dataframe创建与常规操作
DataFrames 为Scala、Java、Python和R 中的结构化数据操作提供了一种特定于领域的语言。
如上所述，在 Spark 2.0 中，DataFrame 只是RowScala 和 Java API中s 的Dataset 。与强类型 Scala/Java 数据集附带的“类型转换”相比，这些操作也称为“无类型转换”。
这里我们包括一些使用 Datasets 进行结构化数据处理的基本示例：

使用SparkSession，应用程序可以从现有的RDD、Hive 表或Spark 数据源创建数据帧。
例如，以下内容基于 JSON 文件的内容创建一个 DataFrame：

In [65]:
import org.apache.spark.sql.SparkSession
import spark.implicits._

import org.apache.spark.sql.SparkSession
import spark.implicits._


In [4]:
val spark = SparkSession
  .builder()
  .appName("Spark SQL basic example")
  .config("spark.some.config.option", "some-value")
  .getOrCreate()


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@11edb250


In [58]:
//生成一个dataFrame
val peopleDF= Seq(
     ("Michael","null"),
     ("Andy","30") ,
     ("Justin","19")
    ).toDF("name","age")

peopleDF.show()


+-------+----+
|   name| age|
+-------+----+
|Michael|null|
|   Andy|  30|
| Justin|  19|
+-------+----+



peopleDF: org.apache.spark.sql.DataFrame = [name: string, age: string]


In [6]:
val df = spark.read.json("people.json")

// Displays the content of the DataFrame to stdout
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



df: org.apache.spark.sql.DataFrame = [age: bigint, name: string]


In [73]:
//从csv文件加载dataFrame
spark.read.option("header", true).csv("people.csv").show()

+---------+----+
|     name| age|
+---------+----+
|  Michael|null|
|     Andy|  30|
|   Justin|  19|
+---------+----+



对DataFrame进行操作

In [14]:
//以树结构打印Schema
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [15]:
//选择多列
df.select(df("name"),df("age")+1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [16]:
//条件过滤
df.filter(df("age") > 20 ).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [17]:
//分组聚合
df.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



In [18]:
//排序
df.sort(df("age").desc).show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



In [19]:
//多列排序
df.sort(df("age").desc, df("name").asc).show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



In [20]:
//对列进行重命名
df.select(df("name").as("username"),df("age")).show()

+--------+----+
|username| age|
+--------+----+
| Michael|null|
|    Andy|  30|
|  Justin|  19|
+--------+----+



以编程方式运行sql查询

In [33]:
//Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")
val sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



sqlDF: org.apache.spark.sql.DataFrame = [age: bigint, name: string]


In [66]:
case class Person(name: String, age: Long)

defined class Person


创建数据集

In [69]:

// Encoders are created for case classes
val caseClassDS = (Seq(Person("Andy", 32))).toDS()
caseClassDS.show()


// Encoders for most common types are automatically provided by importing spark.implicits._
//val primitiveDS = Seq(1, 2, 3).toDS()
//primitiveDS.map(_ + 1).collect() // Returns: Array(2, 3, 4)

// DataFrames can be converted to a Dataset by providing a class. Mapping will be done by name
val path = "people.json"
val peopleDS = spark.read.json(path).as[Person]
peopleDS.show()

+----+---+
|name|age|
+----+---+
|Andy| 32|
+----+---+

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



caseClassDS: org.apache.spark.sql.Dataset[Person] = [name: string, age: bigint]
path: String = people.json
peopleDS: org.apache.spark.sql.Dataset[Person] = [age: bigint, name: string]


与RDD互操作，将现有的 RDD 转换为数据集。Spark SQL 支持两种不同的方法将现有的 RDD 转换为数据集。
1.第一种方法使用反射来推断包含特定类型对象的 RDD 的模式。
2.通过编程接口，该接口允许您构建架构，然后将其应用于现有 RDD。

In [71]:
//import spark.implicits._    ---用于从 RDD 到 DataFrame 的隐式转换
case class Person(name: String, age: Long)

defined class Person


In [72]:

//从文本文件创建 Person 对象的 RDD，将其转换为 Dataframe
val peopleDF = spark.sparkContext
  .textFile("people.txt")
  .map(_.split(","))
  .map(attributes => Person(attributes(0), attributes(1).trim.toInt))
  .toDF()
//将 DataFrame 注册为临时视图(临时视图，视图的生命周期取决于sparksession)，视图可以实现SparkSession能够以编程方式运行SQL查询并返回结果
peopleDF.createOrReplaceTempView("people")
val teenagersDF = spark.sql("SELECT name, age FROM people WHERE age BETWEEN 13 AND 19")
teenagersDF.map(teenager => "Name: " + teenager(0)).show()
teenagersDF.map(teenager => "Name: " + teenager.getAs[String]("name")).show()
//implicit val mapEncoder = org.apache.spark.sql.Encoders.kryo[Map[String, Any]]
//teenagersDF.map(teenager => teenager.getValuesMap[Any](List("name", "age"))).collect()


org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 50.0 failed 1 times, most recent failure: Lost task 0.0 in stage 50.0 (TID 55) (DESKTOP-URD341V executor driver): java.lang.NoClassDefFoundError: Could not initialize class 

In [46]:


//将现有RDD转为数据集
import org.apache.spark.sql.Row

import org.apache.spark.sql.types._

// Create an RDD of Person objects from a text file, convert it to a Dataframe
val peopleRDD = spark.sparkContext.textFile("people.txt")

// The schema is encoded in a string
val schemaString = "name age"

// Generate the schema based on the string of schema
val fields = schemaString.split(" ")
  .map(fieldName => StructField(fieldName, StringType, nullable = true))
val schema = StructType(fields)

// Convert records of the RDD (people) to Rows
val rowRDD = peopleRDD
  .map(_.split(","))
//.trim获取字段的值转为string
  .map(attributes => Row(attributes(0), attributes(1).trim))

// Apply the schema to the RDD
val peopleDF = spark.createDataFrame(rowRDD, schema)

// Creates a temporary view using the DataFrame
peopleDF.createOrReplaceTempView("people")

// SQL can be run over a temporary view created using DataFrames
val results = spark.sql("SELECT name FROM people")

// The results of SQL queries are DataFrames and support all the normal RDD operations
// The columns of a row in the result can be accessed by field index or by field name
results.map(attributes => "Name: " + attributes(0)).show()


+-------------+
|        value|
+-------------+
|Name: Michael|
|   Name: Andy|
| Name: Justin|
+-------------+



import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
peopleRDD: org.apache.spark.rdd.RDD[String] = people.txt MapPartitionsRDD[222] at textFile at <console>:56
schemaString: String = name age
fields: Array[org.apache.spark.sql.types.StructField] = Array(StructField(name,StringType,true), StructField(age,StringType,true))
schema: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,true), StructField(age,StringType,true))
rowRDD: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[224] at map at <console>:69
peopleDF: org.apache.spark.sql.DataFrame = [name: string, age: string]
results: org.apache.spark.sql.DataFrame = [name: string]
